In [41]:
using DifferentialEquations
using Plots
using DifferentialEquations.EnsembleAnalysis
using DiffEqGPU, OrdinaryDiffEq, StaticArrays, CUDA
using LinearAlgebra

In [95]:
const rx = 8.7f-4
const ry = 1.5f-3
const ax = 1.1f-5
const ay = 1.1f-5
const Ax = 4.7f13
const Ay = 4.7f13
const dx0 = 2f-3
const dy0 = 2f-3
const dx1 = 129
const dy1 = 129
const dy0t = 2f-9
const cxx = 7.5f-5
const cxy = 7.5f-5
const cyx = 7.5f-5
const cyy = 7.5f-5
const es = 2
const rs = 3f-4
const ea = 2f9
const rₘ = 2f-8
const Iₑ = 7


function dimensional_reduced_cancitis_rhs(du, u, p, t)
    # out-of-place version with static vectors
    x0 = view(u, 1:1)
    y0 = view(u, 2:2)
    x1 = ax.*Ax.*x0./dx1
    y1 = ay.*Ay.*y0./dy1
    phix = 1 ./(1 .+(cxx.*x0 + cxy.*y0).^2)
    phiy = 1 ./(1 .+(cyx.*x0 + cyy.*y0).^2)
    kappa = dx0.*x0 .+ (dy0t.*y0 .+ dy0).*y0 .+ dx1.*x1 .+ dy1.*y1
    a = 0.5 .*((Iₑ./rs).^2 .+ 4 .*es.*kappa./(ea.*rs)).^(0.5) .- Iₑ./(2 .*rs)
    s = rs.*a./es .+ Iₑ./es
    du1 = @view du[1:1]
    du2 = @view du[2:2]
    @. du1 = (rx.*phix.*s - dx0 - ax).*x0 - rₘ.*s.*x0
    @. du2 = (ry.*phiy.*s - dy0 - dy0t.*y0 - ay).*y0 + rₘ.*s.*x0
end

function reduced_cancitis_noise(du, u, p, t)
    # out of place version with static vectors
    @. du = u./3f2
end

u0 = CuArray([1.01f4, 0f0])
du0 = CuArray(zeros(Float32, 2))
p = CuArray([8.7f-4, 1.5f-3, 1.1f-5, 1.1f-5, 4.7f13, 4.7f13, 2f-3, 2f-3, 129, 129, 2f-9, 7.5f-5, 7.5f-5, 7.5f-5, 7.5f-5, 2, 3f-4, 2f9, 2f-8, 7])
t_final=365f0*80f0
tspan = (0f0, t_final)
# call the rhs function once to precompile it

(0.0f0, 29200.0f0)

In [97]:
t = 0f0
CUDA.allowscalar(false)
#reduced_cancitis_noise(u0, p, t)
@time dimensional_reduced_cancitis_rhs(du0, u0, p, t)

  0.000568 seconds (705 allocations: 54.156 KiB)


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0007289227

In [98]:
cudaproblem = ODEProblem(dimensional_reduced_cancitis_rhs, u0, tspan, p)
sol = solve(cudaproblem)

LoadError: ArgumentError: cannot take the CPU address of a CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}

In [ ]:

CUDA.allowscalar(false)
prob = SDEProblem{false}(dimensional_reduced_cancitis_rhs, reduced_cancitis_noise, u0, tspan, p)
ens_prob = EnsembleProblem(prob)

In [47]:
@time dot(p, p)

  0.000580 seconds (2 allocations: 32 bytes)


4.418f27

In [77]:
@view du0[1:1]

1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0

In [60]:
CUDA.sqrt(p)

LoadError: MethodError: no method matching sqrt(::CuArray{Float32, 1, CUDA.Mem.DeviceBuffer})
[0mClosest candidates are:
[0m  sqrt([91m::StridedMatrix{var"#s814"} where var"#s814"<:Real[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\dense.jl:778
[0m  sqrt([91m::StridedMatrix{var"#s814"} where var"#s814"<:Complex[39m) at C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.6\LinearAlgebra\src\dense.jl:791
[0m  sqrt([91m::Union{Static.StaticBool{N}, Static.StaticFloat64{N}, Static.StaticInt{N}}[39m) where N at C:\Users\jordan\.julia\packages\Static\tQ45Q\src\Static.jl:434
[0m  ...